### module

In [ ]:
import pymysql.cursors
import pandas as pd
import os
import re

> reference:
<br> https://stackoverflow.com/questions/4998629/split-string-with-multiple-delimiters-in-python

### connection

In [ ]:
PW = "1360"
DBASE = "test"
WIKI = "wiki"
LINK = "link"

In [ ]:
con = pymysql.connect(host = 'localhost', 
                            user = 'root',
                            password = PW,
                            db = DBASE,
                            charset = 'utf8mb4')

In [ ]:
cursor = con.cursor()
pd.read_sql("show tables",con)

### Add tables into Database

In [ ]:
table_schema = "'{}'".format(DBASE)
query = '''
        select table_name from information_schema.tables
        where table_schema = {}        
        '''
pd.read_sql(query.format(table_schema), con)

> given skeleton code
<br> description: execute_sql(file_name) function read a file and execute the sql querys into current connected database
<br> reference: http://blog.naver.com/PostView.nhn?blogId=sw4r&logNo=221121111196&parentCategoryNo=&categoryNo=98&viewDate=&isShowPopularPosts=true&from=search

In [ ]:
def execute_sql(file_name):
    sql_file = open(file_name, 'r').readlines()
    query = ""
    i = 0
    for sql_line in sql_file:
        i += 1
        sql_line = sql_line.strip()
        query += sql_line
        if len(query) > 0:
            if query[len(query) - 1] == ';':
                query = query[:len(query) - 1]
                #print('[DEBUG ] ', query)
                cursor.execute(query)
                query = ""

In [ ]:
query = '''
        select count(*) from information_schema.tables
        where table_schema = %s 
        '''
# pd.read_sql(query.format(table_schema), con)
cursor.execute(query, DBASE)
count = cursor.fetchall()
if(count[0][0] == 0):
    execute_sql("link.sql")
    execute_sql("wiki.sql")
count

### parse schedule

> reference:
*  str type built in functions: https://docs.python.org/3/library/functions.html

In [ ]:
# %pwd
# %cd prj2
# %ls 

In [ ]:
# # dirpath = 'C:\\Users\\유성욱\\jupyter\\2019_spring\\DBHW\\HW2\\prj2'
# dirpath = os.getcwd()
# schedules_path = dirpath + '/prj2.sched'

In [ ]:
# queries = []
# schedules = []
# with open(schedules_path, 'r') as f:
#     while True:
#         line = f.readline()
#         if not line: break
# #         print(line)
#         queries.append(line)
#         if(line[0] == "<"):
#     #         print(query.split("> ")[1])
#             schedules.append(line.split("> ")[1])
#         else:
#     #         print(query)
#             schedules.append(line)
# print(len(queries))
# print(len(schedules))

In [ ]:
def parse_schedules(Path):
    queries = []
    schedules = []
    with open(Path, 'r') as f:
        while True:
            line = f.readline()
            if not line: break
    #         print(line)
            queries.append(line)
            if(line[0] == "<"):
        #         print(query.split("> ")[1])
                schedules.append(line.split("> ")[1])
            else:
        #         print(query)
                schedules.append(line)
    return schedules, queries 

In [ ]:
command = input()
# -run prj2.sched
# -run sample_schedules.sched
dirpath = os.getcwd()
if command[0] == "-":
    schedules_path = dirpath + "/"+command.split("-run ")[1]
    if os.path.isfile(schedules_path):
        [schedules, queries] = parse_schedules(schedules_path)
    else:
        print("such file is not exist")

In [ ]:
# schedules
# queries

### excute schedule

In [ ]:
queries
# for query in queries:
# #     implement here
#     if query == condition
#         excute the query

In [ ]:
queries[0][5:]

> execute $<$Transaction ID$>$ [SQL]

In [ ]:
# sample
# print(queries[0])
# print(schedules[0])
print(queries[3])
print(schedules[3])

In [ ]:
if queries[3][0] == "<":
    print("yes")

In [ ]:
pd.read_sql("select * from test", con)

In [ ]:
# sql = "INSERT INTO test (name, id) VALUES ('jenny', 2)"
# sql = "update test set name = 'jenny'  where id = 2"
# sql = "delete from test where id = 2;"
# sql = "drop table test"

In [ ]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)
pd.read_sql("select * from test", con)

In [ ]:
queries[3]

In [ ]:
if(queries[3][0] == "<"):
    print(queries[3].split("> ")[1])

In [ ]:
# q = "<T1> delete from test where id = 3;"
# q = "<T1> update test set text = 'Christiano Ronaldo Howoo Ceremony' where id = 24031233;"
q = "<T2> UPDATE wiki SET text = 'Les Humphries Singers Sing Sang Song Germany 1976' WHERE id = 6241635;"

In [ ]:
q.split("<")[1].lower()

In [ ]:
re.split("> |set |where ",q.split("<")[1].lower())

In [ ]:
# [tran, command_table, col_value, key_value] = re.split("> |set |where ",q.split("<")[1])
[tran, command_table, col_value, key_value] = re.split("> |set |where ",q.split("<")[1].lower())
print([tran, command_table, col_value, key_value])
print(command_table.split(" "))
[command, table, _] = command_table.split(" ")
print(re.split(" = ",col_value))
[col, col_v] = re.split(" = ",col_value)
print(re.split(" = |;",key_value))
[key, key_v, _] = re.split(" = |;",key_value)

In [ ]:
q[5:5+6]

In [ ]:
# q = "<T1> delete from test where id = 3;"
q = "<T1> DELETE FROM wiki WHERE id = 339712;"

In [ ]:
[tran, command, table, key_value] = re.split("> |from |where ",q.split("<")[1])
print([tran, command, table, key_value])
print(re.split(" = |;",key_value))

In [ ]:
q = "<T1> commit; \n"
re.split(">",q.split("<")[1])

In [ ]:
#design write search log function 
def write_transaction_log(dirpath, SQL):
    # parse transaction
    log = ""
    if SQL[5:5+6] == "commit":
        log = "<{0}> commit\n".format(re.split(">",SQL.split("<")[1])[0])
#         print(log)
    elif SQL[5:5+6] == "DELETE":
        [tran, command, table, key_value] = re.split("> |FROM |WHERE ",SQL.split("<")[1]) 
        [key, key_v, _] = re.split(" = |;",key_value)
        log = "<{0}>, <{1}>.<{2}>, <{3}>, \n".format(tran,table,key,key_v)
#         print(log)
    elif SQL[5:5+6] == "UPDATE":    
        [tran, command_table, col_value, key_value] = re.split("> |SET |WHERE ",SQL.split("<")[1])
        [command, table, _] = command_table.split(" ")
        [col, col_v] = re.split(" = ",col_value)
        [key, key_v, _] = re.split(" = |;",key_value)
        # col_oldv = get_old_value
        with con.cursor() as cursor:
            cursor.execute("select {0} from {1} where {2} = {3}".format(col, table, key, key_v))
            old_v = cursor.fetchall()[0][0]
#             print(old_v[0][0])
        log = "<{0}>, <{1}>.<{2}>.<{3}>, <{4}>, <{5}>\n".format(tran,table,key_v,col,old_v,col_v )
#         print(log)
    elif SQL[5:5+8] == "rollback":
        log = "<{0}> abort\n".format(re.split(">",SQL.split("<")[1])[0])
#         print(log)
    log_path = dirpath + "/prj2.log"
    with open(log_path,'a') as f:
        f.write(log)

In [ ]:
key = "id"
table = "test"
key_v = 3

In [ ]:
with con.cursor() as cursor:
#     cursor.execute("select id from test where id = 3 " )
    cursor.execute("select {0} from {1} where {2} = {3}".format(key, table, key, key_v))
    old_v = cursor.fetchall()

In [ ]:
with con.cursor() as cursor:
#     cursor.execute("select id from test where id = 3 " )
    cursor.execute("select * from test")
    result = cursor.fetchall()
    print(result)

In [ ]:
with con.cursor() as cursor:
#     cursor.execute("select id from test where id = 3 " )
    cursor.execute("update test set name = 'wook' where id = 1")
    result = cursor.fetchall()
    print(result)

In [ ]:
old_v[0][0]

In [ ]:
SQL = "<T1> DELETE FROM link WHERE id_from = 339712;"
print(SQL.split("<")[1])
print(re.split("> |FROM | WHERE ",SQL.split("<")[1]))
[tran, command, table, key_value] = re.split("> |FROM | WHERE ",SQL.split("<")[1]) 
[key, key_v, _] = re.split(" = |;",key_value)
log = "<{0}>, <{1}>.<{2}>, <{3}> \n".format(tran,table,key,key_v)
log

In [ ]:
# SQL = "<T1> delete from test where id = 2;"
SQL = "<T1> DELETE FROM link WHERE id_from = 339712;"
# SQL = "<T1> update test set name = 'saewook' where id = 1;"
# SQL = "<T1> commit"
# SQL = "<T1> rollback"
write_transaction_log(dirpath, SQL)

In [ ]:
queries

In [ ]:
for query in queries:
    if query[0] == "<":
        print(query)
        write_transaction_log(dirpath, query)

In [ ]:
open('prj2.log', 'w').close()
%ls

make transaction directory 

In [ ]:
active_transactions = set()

In [ ]:
type(active_transactions)

In [ ]:
queries[9][1:3]

In [ ]:
queries[9][5:5+6]

In [ ]:
active_transactions.add('T3')

In [ ]:
if "T1 "not in active_transactions:
    active_transactions.add('T1')
if queries[9][5:5+6] == "commit":
    active_transactions.remove(queries[9][1:3])

In [ ]:
active_transactions

In [ ]:
log_path = dirpath + "/prj2.log"
log_path

In [ ]:
# queries
# read_file(log_path)

In [ ]:
active_transactions = set()
for schedule in queries:
    if schedule[0] == "<":
        print(schedule)
        if schedule[1:3] not in active_transactions:
            active_transactions.add(schedule[1:3])
        elif schedule[5:5+6] == "commit":
            active_transactions.remove(schedule[1:3])
        elif schedule[5:5+8] == "rollback":
            active_transactions.remove(schedule[1:3])
        print(active_transactions)
        print()
    else:
        print(schedule)
        if schedule[:10] == "checkpoint":
#             print(len(active_transactions))
            print("checkpoint",end="")
            i = 0
            for tran in sorted(active_transactions):
                if( i != len(active_transactions) - 1):
                    print("<",tran,">,",end="")
                else:
                    print("<",tran,">",)
                i += 1

In [ ]:
sorted(set())

In [ ]:
for schedule in queries:
    print(schedule)

In [ ]:
trans = 'T1'
trans

In [ ]:
transaction_table[trans] = int(trans[1])

In [ ]:
transaction_table

In [ ]:
trans in transaction_table

In [ ]:
transaction_table["T1"]

In [ ]:
cursor_dict = {}

In [ ]:
cursor_dict["T1"] = con.cursor() 

In [ ]:
cursor_dict

> execute search $<$keyword$>$
<br>reference:
<br>https://wikidocs.net/26

In [ ]:
def read_file(Path):
    with open(Path, 'r') as f:
        while True:
            line = f.readline()
            if not line: break
            print(line)

In [ ]:
# sample query
queries[12]

In [ ]:
if queries[12][:6] == "search":
    search_query = queries[12].split("search ")[1]
    print("execute search")
    print(search_query)

In [ ]:
search_path = dirpath + "/search.txt"

In [ ]:
sample_search_result = """
id        title                                             TF-IDF              PageRank  
44392605  President_of_the_Quorum_of_the_Twelve               2.34e-05            7.28e-04
34999343  Barry_Mendelson                                     2.34e-05            3.24e-04
28473028  Anti-American_sentiment_in_Pakistan                 2.34e-05            1.85e-04
22553537  Yousef_Pashtun                                      2.61e-05            1.44e-04
13601579  Quentin_L._Cook                                     2.34e-05            1.48e-04
31002740  Westye_Parr_Egeberg                                 2.53e-05            1.36e-04
28246228  St._Louis_Jewish_Light                              2.69e-05            1.02e-04
37619696  Falconar_Avia                                       3.12e-05            8.75e-05
6301341   Colina%2C_Chile                                     2.76e-05            8.93e-05
48652315  Presidency_of_Boris_Yeltsin                         2.34e-05            1.04e-04
"""

In [ ]:
print(sample_search_result)

In [ ]:
num_of_search = 0
open('search.txt', 'w').close()
%ls

In [ ]:
with open(search_path,'a') as f:
    f.write("search {0}\n".format(num_of_search))
    f.write("query {0}\n".format(search_query))
    f.write(sample_search_result+"\n")
    num_of_search += 1

In [ ]:
read_file(search_path)

In [ ]:
#design write search log function 
def write_search_log(num_of_search, search_query, search_result):
    search_path = dirpath + "/search.txt"
    with open(search_path,'a') as f:
        f.write("search {0}\n".format(num_of_search))
        f.write("query {0}\n".format(search_query))
        f.write(search_result+"\n")
        num_of_search += 1

In [ ]:
# con.close()

### Test

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
%run main.py

concurrency control

<br> with out transaction
transaction 없이 sql을 사용하면 interleaved된 사용자 사이에서 anomaly가 발생 할 수 있다. 
<br> with transaction
* start transaction을 하면 다른 cursor에서 반영된 결과가 보이지 않는다. 
* concurrecny 상황 시(동시에 2개의 cursor가 실행중이며 transaction이 lock이되었을 경우)에 현재 실행중인 transaction을 commit해야
    다음 transaction을 실행 할 수 있다.
    * 즉, mysql workbench 에서 
    <br>start transaction;
    <br>UPDATE test SET name = 'swyoo' WHERE id = 1;
    <br>위의 상황에서 밑의 연결을 시도한다면 lock이 걸리고, 밑의 commit을 실행해야 lock이 풀린다
    <br>commit;

In [ ]:
with con.cursor() as cursor:
#         # Create a new record
#         sql = "INSERT INTO `users` (`email`, `password`) VALUES (%s, %s)"
#         cursor.execute(sql, ('webmaster@python.org', 'very-secret'))
    print(pd.read_sql("show tables",con))
    # connection is not autocommit by default. So you must commit to save
    # your changes.
    print("start trans: ", cursor.execute("start transaction"))
#     cursor.execute("INSERT INTO test (id, name) VALUES (3, 'gawon')")
#     cursor.execute("INSERT INTO test (id, name) VALUES (4, 'wook')")
    cursor.execute("update test set name = 'bob' where id = 1")
    print("num of record:", cursor.execute("select * from test"))
    result = cursor.fetchall()
    print(result)
    print("commit: ", cursor.execute("commit"))
    con.commit()

In [ ]:
# con2 = pymysql.connect(host = Connection_Info[0], 
#                             user = Connection_Info[1],
#                             password = Connection_Info[2],
#                             db = Connection_Info[3],
#                             charset = 'utf8mb4')

In [ ]:
# cursor2 = con2.cursor()

In [ ]:
# print("start trans: ", cursor2.execute("start transaction"))
# cursor2.execute("update test set name = 'swyoo' where id = 1")
# print("num of record:", cursor2.execute("select * from test"))
# result = cursor2.fetchall()
# print(result)

In [ ]:
# print("commit: ", cursor2.execute("commit"))
# con2.commit()

In [ ]:
with con.cursor() as cursor:
    print("SHOW PROCESSLIST: ", cursor.execute("show processlist"))
    result = cursor.fetchall()
    print(result)
    con.commit()

In [ ]:
cursor